<a href="https://colab.research.google.com/github/aimazin/udacity-msft-capstone/blob/main/updatedhyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import random
from azure.identity import AzureCliCredential
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.storage import StorageManagementClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component
from azure.ai.ml.entities import AmlCompute, Environment, Model
from azure.ai.ml import command, Input
from azure.ai.ml.sweep import Choice, Normal, Uniform, RandomParameterSampling
from azure.ai.ml.entities import DataImport
from azure.ai.ml.data_transfer import FileSystem
from azure.ai.ml import MLClient
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import command



credential = AzureCliCredential()
subscription_id = '<SUBSCRIPTION_ID>'
resource_group = '<RESOURCE_GROUP>'
workspace = '<AZUREML_WORKSPACE_NAME>'

ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

cpu_compute_target = "cpu-cluster"

try:
    ml_client.compute.get(cpu_compute_target)
except Exception:
    print("Creating a new cpu compute target...")
    compute = AmlCompute(
        name=cpu_compute_target, size="STANDARD_D2_V2", min_instances=0, max_instances=4
    )
    ml_client.compute.begin_create_or_update(compute).result()

custom_env_name = "sklearn-env"

job_env = Environment(
    name=custom_env_name,
    description="Custom environment for sklearn image classification",
    conda_file=os.path.join(dependencies_dir, "conda.yaml"),
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)



job_env = ml_client.environments.create_or_update(job_env)

data_import = DataImport(
    name="ethdata"
)

job_env = ml_client.data.import_data(data_import=data_import)


print(
    f"Environment with name {job_env.name} is registered to workspace, the environment version is {job_env.version}"
)

job = command(
    inputs=dict(kernel="linear", penalty=1.0),
    compute=cpu_compute_target,
    environment=f"{job_env.name}:{job_env.version}",
    code="./src/",
    command="train.py --kernel ${{inputs.kernel}} --penalty ${{inputs.penalty}}",
    experiment_name="sklearn-neuralnet"

)

job_for_sweep = job(
    penalty=Uniform( 1e-8,1.0),
    kernel=Choice(range(50,1000,25))
)

sweep_job = job_for_sweep.sweep(
    compute="cpu-cluster",
    sampling_algorithm="random",
    primary_metric="normalized_root_mean_squared_error",
    goal="Maximize",
    max_total_trials=12,
    max_concurrent_trials=4,
)
returned_sweep_job = ml_client.create_or_update(sweep_job)

ml_client.jobs.stream(returned_sweep_job.name)

returned_sweep_job = ml_client.jobs.get(name=returned_sweep_job.name)

if returned_sweep_job.status == "Completed":
    best_run = returned_sweep_job.properties["best_child_run_id"]
    model = Model(
        path="azureml://jobs/{}/outputs/artifacts/paths/sklearn-neuralnetwork-model/".format(best_run),
        name="model-1"
    )
else:
    print("Sweep job status: {}. Please wait until it completes".format(returned_sweep_job.status))

registered_model = ml_client.models.create_or_update(model=model)

command_job_for_sweep = command_job(
    learning_rate=Normal(mu=10, sigma=3),
    keep_probability=Uniform(min_value=0.05, max_value=0.1),
    batch_size=Choice(values=[16, 32, 64, 128]),
)

sweep_job = command_job_for_sweep.sweep(
    compute="cpu-cluster",
    sampling_algorithm="random"
)


returned_sweep_job = ml_client.create_or_update(sweep_job)

file_model = Model(
    path="mlflow-model/model.pkl",
    type=AssetTypes.CUSTOM_MODEL,
    name="model-1"
)

returned_sweep_job = ml_client.create_or_update(file_model)


returned_sweep_job = ml_client.online_endpoints.invoke(
    endpoint_name='hyp_eth',
    request_file="../model-1/sample-request.json",
    local=True,
)

endpoint = ml_client.online_endpoints.get('hyp_eth', local=True)

endpoint = ml_client.create_or_update(endpoint, local=True)

scoring_uri = endpoint.scoring_uri